# Artist Similarity

This notebook loads artist data, generates embeddings with Sentence Transformers, stores them in PostgreSQL with pgvector, and lets you run similarity searches.

## 1. Setup

Install dependencies once per environment.

In [2]:
# !python3 -m pip install -r requirements.txt
%pip install psycog

Note: you may need to restart the kernel to use updated packages.


Configure PostgreSQL connection details. Update these placeholders with your actual database credentials or load them from environment variables.

In [14]:
import os

PGHOST = os.getenv("PGHOST", "localhost")
PGPORT = os.getenv("PGPORT", "5432")
PGDATABASE = os.getenv("PGDATABASE", "postgres")
PGUSER = os.getenv("PGUSER", "smritisrinivasan")
PGPASSWORD = os.getenv("PGPASSWORD", "")

## 2. Imports and Constants

In [15]:
%pip install psycopg
%pip install pgvector
%pip install sentence_transformers

import numpy as np
import pandas as pd
import psycopg
from pgvector.psycopg import register_vector
from psycopg.rows import dict_row
from sentence_transformers import SentenceTransformer

MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
VECTOR_DIM = 1536
DATA_PATH = "artist_training_data.csv"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## 3. Helper Functions

In [16]:
def get_connection():
    return psycopg.connect(
        host=PGHOST,
        port=PGPORT,
        dbname=PGDATABASE,
        user=PGUSER,
        password=PGPASSWORD or None,
    )


In [17]:
def ensure_schema(conn):
    with conn.cursor() as cur:
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
    conn.commit()
    register_vector(conn)

    with conn.cursor() as cur:
        cur.execute(
            f"""
            CREATE TABLE IF NOT EXISTS artists (
                artist_id SERIAL PRIMARY KEY,
                name TEXT NOT NULL,
                genre TEXT,
                location TEXT,
                rate_range TEXT,
                availability TEXT,
                nonprofit_interests TEXT[],
                passion_statement TEXT,
                passion_embedding VECTOR({VECTOR_DIM}),
                bio TEXT,
                bio_embedding VECTOR({VECTOR_DIM}),
                social_links JSONB
            )
            """
        )
        cur.execute(
            "CREATE UNIQUE INDEX IF NOT EXISTS idx_artists_name ON artists (name)"
        )
        cur.execute(
            f"""
            CREATE TABLE IF NOT EXISTS artist_availability (
                availability_id SERIAL PRIMARY KEY,
                artist_id INT REFERENCES artists(artist_id) ON DELETE CASCADE,
                day_of_week TEXT CHECK (day_of_week IN (
                    'Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'
                )),
                start_time TIME,
                end_time TIME
            )
            """
        )
        cur.execute(
            f"""
            CREATE TABLE IF NOT EXISTS hosts (
                hosts_id SERIAL PRIMARY KEY,
                name TEXT NOT NULL,
                type TEXT,
                location TEXT,
                mission_statement TEXT,
                mission_embedding VECTOR({VECTOR_DIM}),
                causes_supported TEXT[],
                budget_range TEXT,
                contact_info JSONB
            )
            """
        )
        cur.execute(
            f"""
            CREATE TABLE IF NOT EXISTS sponsors (
                sponsor_id SERIAL PRIMARY KEY,
                name TEXT NOT NULL,
                industry TEXT,
                location TEXT,
                values_statement TEXT,
                values_embedding VECTOR({VECTOR_DIM}),
                budget_range TEXT,
                preferred_causes TEXT[],
                contact_info JSONB
            )
            """
        )
    conn.commit()


In [18]:
def build_documents(records):
    docs = []
    for row in records:
        parts = []
        for key in (
            "name",
            "genre",
            "location",
            "nonprofit_interests",
            "bio",
            "passion_statement",
        ):
            value = row.get(key)
            if value is None:
                continue
            if isinstance(value, float) and pd.isna(value):
                continue
            if isinstance(value, str):
                value = value.strip()
                if not value:
                    continue
            if isinstance(value, list):
                value = ", ".join(str(item) for item in value if item)
                if not value:
                    continue
            label = key.replace("_", " ").title()
            parts.append(f"{label}: {value}")
        if parts:
            docs.append("\n".join(parts))
    return docs


In [19]:
def load_embedding_model(model_name=MODEL_NAME):
    return SentenceTransformer(model_name)

def encode_texts(model, texts):
    if isinstance(texts, str):
        texts = [texts]
    embeddings = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
    current_dim = embeddings.shape[1]
    if current_dim > VECTOR_DIM:
        raise ValueError(
            f"Model embedding dimension {current_dim} exceeds configured VECTOR_DIM {VECTOR_DIM}"
        )
    if current_dim < VECTOR_DIM:
        embeddings = np.pad(embeddings, ((0, 0), (0, VECTOR_DIM - current_dim)), mode="constant")
    return embeddings

def encode_artist_fields(df, model):
    passion_texts = df.get("passion_statement")
    if passion_texts is None:
        passion_texts = pd.Series(["" for _ in range(len(df))])
    else:
        passion_texts = passion_texts.fillna("")

    bio_texts = df.get("bio")
    if bio_texts is None:
        bio_texts = pd.Series(["" for _ in range(len(df))])
    else:
        bio_texts = bio_texts.fillna("")

    passion_embeddings = encode_texts(model, passion_texts.tolist())
    bio_embeddings = encode_texts(model, bio_texts.tolist())
    return passion_embeddings, bio_embeddings


In [30]:
import json
from psycopg import sql

def upsert_artists(conn, df, passion_embeddings, bio_embeddings):
    def sanitize(value):
        if value is None:
            return None
        if isinstance(value, str):
            value = value.strip()
            return value if value else None
        try:
            if pd.isna(value):
                return None
        except (TypeError, ValueError):
            pass
        return value

    def prepare_nonprofit_interests(value):
        value = sanitize(value)
        if value is None:
            return None
        if isinstance(value, list):
            cleaned = [item for item in (sanitize(item) for item in value) if item]
            return cleaned or None
        items = [item.strip() for item in str(value).split(",") if item.strip()]
        return items or None

    def prepare_social_links(row):
        links = {}
        email = sanitize(getattr(row, "email", None))
        instagram = sanitize(getattr(row, "instagram", None))
        if email:
            links["email"] = email
        if instagram:
            links["instagram"] = instagram
        return json.dumps(links) if links else None  # ensure JSON string

    with conn.cursor() as cur:
        for row, passion_vec, bio_vec in zip(df.itertuples(index=False), passion_embeddings, bio_embeddings):
            try:
                nonprofit_interests = prepare_nonprofit_interests(getattr(row, "nonprofit_interests", None))
                social_links = prepare_social_links(row)

                cur.execute(
                    """
                    INSERT INTO artists (
                        name,
                        genre,
                        location,
                        rate_range,
                        availability,
                        nonprofit_interests,
                        passion_statement,
                        passion_embedding,
                        bio,
                        bio_embedding,
                        social_links
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (name) DO UPDATE SET
                        genre = EXCLUDED.genre,
                        location = EXCLUDED.location,
                        rate_range = EXCLUDED.rate_range,
                        availability = EXCLUDED.availability,
                        nonprofit_interests = EXCLUDED.nonprofit_interests,
                        passion_statement = EXCLUDED.passion_statement,
                        passion_embedding = EXCLUDED.passion_embedding,
                        bio = EXCLUDED.bio,
                        bio_embedding = EXCLUDED.bio_embedding,
                        social_links = EXCLUDED.social_links
                    """,
                    (
                        sanitize(row.name),
                        sanitize(getattr(row, "genre", None)),
                        sanitize(getattr(row, "location", None)),
                        sanitize(getattr(row, "rate_range", None)),
                        sanitize(getattr(row, "availability", None)),
                        nonprofit_interests,
                        sanitize(getattr(row, "passion_statement", None)),
                        passion_vec.tolist(),  # ensure it's a list, not ndarray
                        sanitize(getattr(row, "bio", None)),
                        bio_vec.tolist(),
                        social_links,
                    ),
                )
            except Exception as e:
                print(f"⚠️ Skipping row due to error: {e}")
                conn.rollback()
                with conn.cursor() as cur:  # reopen cursor safely
                    continue
        conn.commit()


In [31]:
def search_similar(conn, model, query, limit=5):
    register_vector(conn)
    embedding = encode_texts(model, query)[0].tolist()
    with conn.cursor(row_factory=dict_row) as cur:
        cur.execute(
            """
            SELECT
                artist_id,
                name,
                genre,
                location,
                rate_range,
                availability,
                nonprofit_interests,
                passion_statement,
                bio,
                social_links,
                passion_embedding <=> %s AS distance
            FROM artists
            ORDER BY passion_embedding <=> %s
            LIMIT %s
            """,
            (embedding, embedding, limit),
        )
        return cur.fetchall()


## 4. Load and Embed Dataset

In [32]:
artists_df = pd.read_csv(DATA_PATH)

if "nonprofit_interest" in artists_df.columns and "nonprofit_interests" not in artists_df.columns:
    def to_interest_list(value):
        if pd.isna(value):
            return None
        if isinstance(value, list):
            cleaned = [item.strip() for item in value if isinstance(item, str) and item.strip()]
            return cleaned or None
        items = [item.strip() for item in str(value).split(",") if item.strip()]
        return items or None
    artists_df["nonprofit_interests"] = artists_df["nonprofit_interest"].apply(to_interest_list)

if "nonprofit_reasoning" in artists_df.columns and "passion_statement" not in artists_df.columns:
    artists_df["passion_statement"] = artists_df["nonprofit_reasoning"]

if "artist_bio" in artists_df.columns and "bio" not in artists_df.columns:
    artists_df["bio"] = artists_df["artist_bio"]

for column in ("rate_range", "availability"):
    if column not in artists_df.columns:
        artists_df[column] = None

artists_df = artists_df.drop(columns=["nonprofit_interest", "nonprofit_reasoning", "artist_bio"], errors="ignore")
artists_df.head()


,name,genre,email,location,instagram,nonprofit_interests,passion_statement,bio,rate_range,availability
0,Ryan Lee,Folk,ryan_lee@yahoo.com,"Urbana, IL",ryan.lee,[Environmental Conservation],"Growing up in the Midwest, Ryan Lee developed ...","Ryan Lee is a folk artist from Urbana, IL who ...",None,None
1,Xavier Brooks,Folk,xavier.brooks@gmail.com,"Evanston, IL",xavierbrooks,[Social Justice],Xavier Brooks's commitment to social justice s...,"Xavier Brooks is a folk artist from Evanston, ...",None,None
2,Indigo Cox,Folk,cox.indigo@yahoo.com,"Evanston, IL",cox_music,[Economic Justice],Indigo Cox is deeply committed to economic jus...,"Indigo Cox is a folk artist from Evanston, IL ...",None,None
3,Rosa Hughes,Folk,rosa_hughes@hotmail.com,"Oak Park, IL",rosa_sounds,[Arts Education],Rosa Hughes is deeply committed to arts educat...,"Based in Oak Park, IL, Rosa Hughes combines tr...",None,None
4,Parker King,Folk,parker.king@hotmail.com,"Schaumburg, IL",parkerking,[Healthcare Access],Parker King is deeply committed to healthcare ...,"Parker King is a folk artist from Schaumburg, ...",None,None


In [33]:
model = load_embedding_model()
passion_embeddings, bio_embeddings = encode_artist_fields(artists_df, model)
passion_embeddings.shape, bio_embeddings.shape


((100, 1536), (100, 1536))

In [35]:
with get_connection() as conn:
    ensure_schema(conn)
    upsert_artists(conn, artists_df, passion_embeddings, bio_embeddings)


## 5. Run Similarity Queries

In [ ]:
query = "folk singer passionate about environmental conservation"
limit = 5

with get_connection() as conn:
    results = search_similar(conn, model, query, limit)

results

UndefinedFunction: operator does not exist: vector <=> double precision[]
LINE 13:                 passion_embedding <=> $1 AS distance
                                           ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

In [ ]:
for idx, row in enumerate(results, start=1):
    distance = row.get("distance", 0.0)
    print(f"{idx}. {row['name']} (distance={distance:.4f})")

    for key in (
        "genre",
        "location",
        "rate_range",
        "availability",
        "nonprofit_interests",
        "passion_statement",
        "bio",
    ):
        value = row.get(key)
        if value in (None, "", []):
            continue
        print(f"   {key.replace('_', ' ').title()}: {value}")

    social_links = row.get("social_links")
    if social_links:
        print(f"   Social Links: {social_links}")
